# Machine Learning Pipeline with KubeDirector - Lab 6
## Dynamic? Did someone say a dynamic ML pipeline?

### **Lab workflow**

Over time, your model might get stale because of changing traffic conditions and a changing dataset. You may also want to enhance the performance of your model as a result of monitoring the accuracy of the predictions over time. 

To illustrate the dynamic aspect of the ML pipeline you have just built, let’s imagine you want to improve the prediction accuracy of your model. 

In this lab: 

1. You will **retrain** your model by tuning the model parameters to get better predictive performance and save your retrained model into a new file. 

2. You will then update the model registry information in the configMap kubernetes resource with the file path for the newly trained model file and associated scoring script file. 

3. You will finally make a new prediction query.

### **1- Initialize the environment**

Let's first define the environment variables needed to execute this part of the lab.

In [ ]:
#
# environment variables
#
studentId="student{{ STDID }}" # your Jupyter Notebook student Identifier (i.e.: student<xx>)

gateway_host="{{ HPEECPGWNAME }}"
Internet_access="{{ JPHOSTEXT }}"

kc_secret="kc-secret-students.yaml" #the kubeconfig secret object.
JupyterNotebookApp="cr-cluster-jupyter-notebook.yaml" # the Jupyter Notebook KD App manifest you will deploy to build your model
DeploymentEngineApp="cr-cluster-endpoint-wrapper.yaml" # the Deployment engine KD App manifest you will deploy to query your model for answers 
PipelineConfigMap="ml-pipeline-configmap.yaml" # ConfigMap manifest used to register the trained model version 1 
PipelineConfigMapv2="ml-pipeline-configmap-v2.yaml" # ConfigMap manifest used to register the trained model version 2 
#
clusterName="inference-server-${studentId}"
#
# Model registry information
#
TrainingModel="model-${studentId}"
modelVersion="1"
#
echo "Your studentId is: "$studentId 

### **2- Retrain your model**

#### <font color="red">Go back to your **local Jupyter Notebook**, Lab Part 3, and run the code cell from the step **"6-Retrain the model to improve model accuracy"**.</font>

### **3- Adjust the model registry information**

Once your model is retrained, adjust the model registry information.
Run the code cells below to update the model registry information with the path of your **retrained** model file:

In [ ]:
cat $PipelineConfigMapv2

In [ ]:
kubectl apply -f $PipelineConfigMapv2

**Then, use the command `kubectl describe configmap` below and check the events logged against it:**

In [ ]:
kubectl describe configmap $TrainingModel

In the events section at the bottom of the command output, you should notice the message:  
_"Connected to cluster {your deployment-engine cluster name}; updating it."_ 

>**Note:** KubeDirector has detected the change of the model registry information. KubeDirector then immediately updates the model registry metadata information on the PODs/Containers of your deployment engine cluster. The deployment engine cluster can therefore reference the newly trained model file to use to serve the prediction requests. 

**And use the command `kubectl describe kdcluster` below and check the events logged against it:** 

In the events section at the bottom of the command output, you should notice the message:  

_"connected configmap has changes, updated context for the PODs of your instance of the deployment engine cluster"._ 

In [ ]:
kubectl describe kdcluster $clusterName

>**Note:** By changing the configMap, your entire ML pipeline will be reconciled by KubeDirector operator for you, while the containers of the _deployment engine_ cluster environment remain running. 
In terms of the use case here, that means you can update existing model registry information or add another model to be handled by the deployment engine without interrupting any current requests that the deployment engine environment is processing.   
**The key use of the KubeDirector applications, the KubeDirector clusters, and the KubeDirector Connections capability is what makes your ML pipeline very _dynamic_.**

Let's make another query using your retrained model:

In [ ]:
#
# Getting the Model Serving access point from the haproxy service of the LoadBalancer (role: LoadBalancer, internal port: 32700):
#
LoadBalancerURL=$(kubectl describe service -l kubedirector.hpe.com/kdcluster=${clusterName},kubedirector.hpe.com/role=LoadBalancer | grep gateway/32700 | awk '{print $2}')
LoadBalancerPort=$(echo $LoadBalancerURL | cut -d':' -f 2) # extract the gateway re-mapped port value.
LoadBalancer_endpoint="https://$gateway_host:$LoadBalancerPort"
LoadBalancerAuthToken=$(kubectl describe service -l kubedirector.hpe.com/kdcluster=${clusterName},kubedirector.hpe.com/role=LoadBalancer | grep kd-auth-token  | awk '{print $2}' | tr -d '\r')
#
# REST API query:
#
curl --location -k -s --request POST "${LoadBalancer_endpoint}/${TrainingModel}/${modelVersion}/predict" \
--header "X-Auth-Token: ${LoadBalancerAuthToken}" \
--header 'Content-Type: application/json' \
--data-raw '{
    "use_scoring": true,
    "scoring_args": {
        "work": 0,
        "start_latitude": 40.57689727,
        "start_longitude": -73.99047356,
        "end_latitude": 40.72058154,
        "end_longitude": -73.99740673,
        "distance": 8,
        "weekday": 1,
        "hour": 9,
        "month_1": 0,
        "month_2": 1,
        "month_3": 0,
        "month_4": 0,
        "month_5": 0,
        "month_6": 0
    }
}' | python -m json.tool | grep output | cut -d'\' -f 3 | cut -c 2-

### **4- Time to go through some cleanup**

Delete your deployment engine, configMap, and your local Jupyter Notebook.

In [ ]:
kubectl delete -f $kc_secret
rm -f $kc_secret

In [ ]:
kubectl delete -f $DeploymentEngineApp

In [ ]:
kubectl delete -f $PipelineConfigMap

In [ ]:
kubectl delete -f $JupyterNotebookApp

## Summary

In this lab, you learned how the key use of KubeDirector applications, the KubeDirector clusters, and the KubeDirector Connections capability is what makes your ML pipeline very **dynamic**.

* [Conclusion](7-Conclusion.ipynb)